In [18]:
# 如何抓取三大法人買賣金額統計表
import requests
res = requests.get('http://www.twse.com.tw/ch/trading/fund/BFI82U/BFI82U.php?report1=day&input_date=105%2F01%2F12&mSubmit=%ACd%B8%DF&yr=2016&w_date=20160111&m_date=20160101')
res.encoding = 'big5'
#print res.text

In [17]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')
for tr in soup.select('.board_trad tr')[2:]:
    for td in tr.select('td'):
        print(td.text)


自營商(自行買賣)
1,026,801,020
1,581,378,505
-554,577,485
自營商(避險)
4,880,014,670
6,405,478,792
-1,525,464,122
投信
1,780,218,900
1,055,485,865
724,733,035
外資及陸資
22,814,489,457
26,000,602,857
-3,186,113,400
合計
30,501,524,047
35,042,946,019
-4,541,421,972


In [13]:
# 產生日期序列
from datetime import date, timedelta

today = date.today()
for i in range(1, 10):
    today = today + timedelta(days = -1)
    dayary = str(today).split('-')
    print '-'.join([str(int(dayary[0]) - 1911), dayary[1], dayary[2]])

105-01-13
105-01-12
105-01-11
105-01-10
105-01-09
105-01-08
105-01-07
105-01-06
105-01-05


In [45]:
# 如何抓取三大法人買賣金額統計表(多日)
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta

url = 'http://www.twse.com.tw/ch/trading/fund/BFI82U/BFI82U.php?report1=day&input_date={0}&mSubmit=%ACd%B8%DF&yr=2016&w_date=20160111&m_date=20160101'

def money_conversion(input_ele):
    return int(''.join(input_ele.split(',')))

def getTradeValue(today):
    dayary = str(today).split('-')
    dt = '%2F'.join([str(int(dayary[0]) - 1911), dayary[1], dayary[2]])
    res = requests.get(url.format(dt))
    res.encoding = 'big5'
    #print(res.text)
    soup = BeautifulSoup(res.text, 'html5')
    for tr in soup.select('.board_trad tr')[2:]:
        td  = tr.select('td')
        print(u"{0} {1} {2} {3} {4}".format(td[0].text, td[1].text, td[2].text, td[3].text, today)) # unicode issue



today = date.today()
for i in range(1, 2):
    today = today + timedelta(days = -1)
    getTradeValue(today)


自營商(自行買賣) 654,724,190 1,179,338,200 -524,614,010 2016-01-13
自營商(避險) 4,592,655,640 5,339,293,473 -746,637,833 2016-01-13
投信 763,855,500 788,404,135 -24,548,635 2016-01-13
外資及陸資 27,017,823,458 25,868,276,133 1,149,547,325 2016-01-13
合計 33,029,058,788 33,175,311,941 -146,253,153 2016-01-13


In [3]:
import sqlite3 as lite
con = lite.connect('finance.sqlite')
cur = con.cursor()
cur.execute("select * from InvestorTradingValue")
ret = cur.fetchone()
print ret
#cur.execute("insert into InvestorTradingValue(item, total_buy, total_sell, difference, date) values('foreign', 20, 30, -10, '2016-01-13')")
#con.commit()
con.close()

(1, u'foreign', 20, 30, -10, u'2016-01-13')


In [10]:
# 抓取個股日本益比csv
import requests
from bs4 import BeautifulSoup
import base64
import shutil

res = requests.get('http://www.twse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php')

res.encoding = 'big5'
#print(res.text)
soup = BeautifulSoup(res.text, 'html5')  # use html5lib parser, looks like html is not formatted, lxml parser will lost some nodes

payload = {}

for inp in soup.select('input'):
    if 'hidden' in inp.get('type'):
        payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8'))

res2 = requests.post('http://www.twse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv', data = payload, stream = True)

f = open('export.csv', 'wb')
shutil.copyfileobj(res2.raw, f)
f.close()
